In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from random import randint

## MNIST자료 불러오기
28*28사이즈의 숫자 이미지를 행렬로 표현하였으며 2차원 구조를 정규화 하여 1차원 구조로 표현되있다.
라벨의 경우 one-hot방식인데 0~9까지의 숫자를 [0,0,0,0,0,1,0,0,0,0]이런 식으로 표현하였다.

In [2]:
mnist = input_data.read_data_sets("./MNIST_data/",one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
print("총 학습용 데이터 갯수 : ",len(mnist.train.labels))
print("총 검증용 데이터 갯수 : ",len(mnist.validation.labels))
print("총 테스트용 데이터 갯수 : ",len(mnist.test.labels))
print("개별 데이터의 행렬 길이 : ",len(mnist.train.images[0]))
print("개별 라벨의 행렬 길이 : ", len(mnist.train.labels[0]))

총 학습용 데이터 갯수 :  55000
총 테스트용 데이터 갯수 :  10000
총 검증용 데이터 갯수 :  5000
개별 데이터의 행렬 길이 :  784
개별 라벨의 행렬 길이 :  10


## 파라미터 설정

In [4]:
x = tf.placeholder(tf.float32,[None,784], name="images")
label = tf.placeholder(tf.float32,[None,10],name="labels")

W = tf.Variable(tf.zeros([784,10]),name="Weight")
b = tf.Variable(tf.zeros([10]),name="bias")

## 히스토그램

In [5]:
W_hist = tf.histogram_summary("Weights",W)
b_hist = tf.histogram_summary("biases",b)
label_hist = tf.histogram_summary("labels",label)

## 가설

In [6]:
with tf.name_scope("hypoyhesize") as scope:
    hypothesize  = tf.nn.softmax(tf.matmul(x,W)+b)

## 비용함수(교차엔트로피)

In [7]:
with tf.name_scope("cost_func") as scope:
    cross_entropy = -tf.reduce_sum(label*tf.log(hypothesize),reduction_indices=1)
    cost = -tf.reduce_mean(label*tf.log(hypothesize)+(1-label)*tf.log(1-hypothesize))
    cost_sum = tf.scalar_summary("cost",cost)

## 학습 단계

In [8]:
with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

## 모델 평가하기

In [9]:
with tf.name_scope("test") as scope:
    real_label = tf.arg_max(label,1) # 실제 라벨
    learn_label = tf.arg_max(hypothesize,1) # 머신러닝 결과
    correct_prodiction = tf.equal(real_label,learn_label) # 둘다 값이 같으면 true, 틀리면 false

    # 정확도
    accuracy = tf.reduce_mean(tf.cast(correct_prodiction,tf.float32))
    accuracy_summary = tf.scalar_summary("accuracy",accuracy)

## 초기화

In [10]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


## 세션

In [11]:
sess = tf.Session()
sess.run(init)

In [12]:
repeat_count = 1000 # 전체 학습 반복횟수
batch_count = 100# 매회 학습자료량

## 텐서보드 설정

In [13]:
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter("./logs/mnist",sess.graph)

## 학습하기

In [14]:
for i in range(repeat_count+1):
    if i % 10 == 0: # 매 10번째마다 정확도 테스트 수행
        feed = {x:mnist.test.images, label: mnist.test.labels}
        summary,_ = sess.run([merged,accuracy],feed_dict=feed)
        writer.add_summary(summary,i)
    else:
        batch_xs,batch_ys = mnist.train.next_batch(batch_count)
        feed = {x: batch_xs,label:batch_ys}
        sess.run(train,feed_dict=feed)

## 테스트 데이터로 검증

In [15]:
print(sess.run(accuracy, feed_dict={x: mnist.test.images, label:mnist.test.labels}))

0.9063
